# Collection: copublications Inria / Organismes à Berlin (Allemagne)
* demande interne Inria (27/03/2025)
* Réalisation du script : Kumar Guha (Data DCIS/Inria)
* Date 31/03/2025

## Choix
* On ne retient que la première affiliation de chaque auteur (pas les niveaux supérieurs : exemple : Boston University School of Medicine et pas Boston University).
* Si un auteur rattaché à une structure française est aussi rattaché à une structure étrangère, on ne retient pas cette structure étrangère dans la décompte des copubliants étrangers. 
* Les publications où les auteurs sont affiliés à des structures ayant pour tutelle un centre Inria sont comptées comme des copublications FR.
* Organisme à Berlin : toute structure dont le nom ou l'adresse contiennent les termes "Berlin" ou "Potsdam".

## Limites et solution adoptée
* Les structures présentes dans le référentiel Aurehal ne sont pas toujours bien renseignées (ville non mentionnée, adresse manquante), il y a peut-être des organismes situés à Berlin qui n'entreront donc pas dans le résultat. Solution : une liste des organismes allemands mais pas identifiés comme étant à Berlin est ajoutée au résultat.

## Étapes
* Extraire les publications des équipes concernées et récupérer les informations de licence.
* identifier les publications dont les auteurs sont affiliés à un organisme étranger (hors France et DOM TOM)
* On crée des listes d'identifiants uniques pour les affiliations FR, Union Européenne et hors UE.
    *  on exclut les organismes étrangers dont les auteurs sont aussi affiliés à une structure FR
    *  on exclut les affiliations en double pour une même publication

## Résultat
* Génération d'un fichier Excel avec : 
    * Liste des copublications avec un organisme berlinois
    * Liste des organismes berlinois copubliants, et des domaines
    * Liste des équipes INRIA copubliantes
    * Liste des domaines des publications et des organismes berlinois copubliants
    * Liste des autres organismes allemands copubliants
    



## Extraction des publications de HAL


In [2]:
###############################################################
## Extraction des publications de HAL
##############################################
import requests
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
from lxml import etree
import gc 
import locale
import re

###############################################################################################
##################### variables à modifier avant lancement du script ##########################
###############################################################################################

###########################################################
# Définir la structure recherchée:
###########################################################
nom_collection = "INRIA2" # il s'agit des publications des équipes Inria de tous les centres.

# Identifiants des structures qui serviront de critère de filtrage des résultats
# Id des Centres Inria dans Aurehal
id_aurehal_de_la_structure = [
    "419153", "104751", "34586", "2497", "1096051", "129671", "104752", "118511", "454310"
]



##########################################
# Définir la période recherchée
###########################################
annee_debut = "2020"
annee_fin = "2024" # indiquer la même année si la recherche porte sur une seule année

############################################
# Définir le pays des organismes recherchés
###########################################
pays_org = "DE"

#####################################################################
##################### script ########################################
#####################################################################
# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y-%m-%d")

## Spécifier le répertoire de log
log_directory = '../log/'
## Créer le répertoire s'il n'existe pas
os.makedirs(log_directory, exist_ok=True)


# Configuration du logger
log_file = date_extraction_current + '__publications_of_the_year_log.txt'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

# Configurer la localisation en français
locale.setlocale(locale.LC_TIME, "French_France.1252")

# La période est définie par les années saisies dans les variables au-dessus du script

periode = "[" + annee_debut + " TO " + annee_fin + "]"

# Fonction permettant de réessayer s'il n'y a pas de réponse
def fetch_with_retry(url, params=None, max_retries=3, delay=2):

    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params, timeout=10)  # Timeout pour éviter les blocages
            
            if response.status_code == 200:
                print(f"réussi")
                return response  # Succès
            
            print(f"⚠️ Tentative {attempt + 1} échouée ({response.status_code}). Nouvelle tentative...")

        except requests.RequestException as e:
            print(f"⏳ Erreur réseau ({e}), tentative {attempt + 1}...")


#### Processus de récupération de la liste des notices présentes dans HAL pour la période spécifiée
# URL de base de l'API
base_url = f"https://api.archives-ouvertes.fr/search/{nom_collection}?"

# Paramètres de la requête : les résultats sont traités un par un en xml-tei
params = {
    "q": f"publicationDateY_i:{periode}",
    # "fq": f"structId_i:{id_aurehal_de_la_structure}",
    "wt": "xml-tei",
    "rows": 500, # La limite est fixée à 500 pour réduire le temps de traitement sans risquer un timeout
    "sort": "docid asc"
}

# https://api.archives-ouvertes.fr/search/INRIA2?q=publicationDateY_i:[2019%20TO%202024]&fq=structId_i:(413916%20OR%20526070%20OR%20526181%20OR%20521735%20OR%20521714)&wt=xml-tei&rows=100&sort=docid%20asc

# Initialisation du cursorMark (qui permet de réitérer la requête jusqu'à la fin des réponses de l'API)
cursor_mark = "*"
previous_cursor_mark = None

# Définition des variables
compteur = 0
compte_publisUps = 0
partenaire = 0
dataex = []
datafr = []
datapubli = []
unique_org_ex = {}
unique_org_fr = {}
# Inclure les DOM-TOM dans les publications FR
France_et_dom_tom_codes = ['FR','GP', 'RE', 'MQ', 'GF', 'YT', 'PM', 'WF', 'TF', 'NC', 'PF']


namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}


#######################################
# La requête est lancée en boucle et obtient un résultat 
# chaque résultat est traité dans la boucle "while"
########################################
while cursor_mark != previous_cursor_mark:
    # Mise à jour du cursorMark
    params["cursorMark"] = cursor_mark
    # print(f"CursorMark: {cursor_mark}")
    compteur += 1
    if compteur % 1 == 0 or compteur == 1:
        print(compteur*500)
        # conversion en dataframes intermédiaires, en cas d'interruption inopinée
        if compteur % 100 == 0:
            # Convertir les données collectées pour les organismes étrangers
            dataex = list(unique_org_ex.values())
            df_ex = pd.DataFrame(dataex)
            # Convertir les données collectées pour les organismes français
            datafr = list(unique_org_fr.values())
            df_fr = pd.DataFrame(datafr)

            # Pour contrôle Liste des publications/logiciels de HAL
            # df_publis = pd.DataFrame(datapubli)
            # df_ex.to_excel(f"df_ex_{compteur}.xlsx", index=False)
            # df_fr.to_excel(f"df_fr_{compteur}.xlsx", index=False)
            # df_publis.to_excel(f"df_publis_{compteur}.xlsx", index=False)
        time.sleep(2)  # Pause de 2 secondes entre les requêtes
   
    # Limite pour tests
    # if compteur > 15:
    #     break

    response = fetch_with_retry(base_url, params)
    if response:
        try:
            tree = etree.fromstring(response.content)
        except etree.XMLSyntaxError:
            print("Erreur de syntaxe XML. Réponse non analysée.")
            continue

        # Récupération de la valeur de next dans l'attribut de la première balise TEI
        next_cursor_mark = tree.attrib.get("next")
        print(f"prochain curseur :{next_cursor_mark}")
        
        # indication du nombre de notices répondant à la requête
        quantity_value = tree.find('.//tei:measure', namespaces=namespaces).attrib.get('quantity')
        
        if cursor_mark == "*":
            # seulement lors de la première boucle, on indique le nombre total de notices répondant à la requête
            print(f"nbre résultats : {quantity_value}. Durée estimée pour 4000 notices : 20 mn")

  

        # TRAITEMENT DES AFFILIATIONS contenues dans la notice
        orgs = tree.findall('.//tei:listOrg[@type="structures"]/tei:org', namespaces=namespaces)
        # print(f"je traite les ORG")
        for org in orgs:
            xml_id = org.xpath('@xml:id', namespaces=namespaces) # code de la structure
            lenom = org.xpath('.//tei:orgName/text()', namespaces=namespaces)
            lacronyme = org.xpath('.//tei:orgName[@type="acronym"]/text()', namespaces=namespaces)
            lepays = org.xpath('.//tei:country/@key', namespaces=namespaces)
            ladresse = [addr.text for addr in org.xpath('.//tei:addrLine', namespaces=namespaces) if addr.text]
            ladresse_value = " ".join(ladresse)
            lesrelations = org.xpath('.//tei:listRelation/tei:relation/@active', namespaces=namespaces) # codes des structures parentes

            # Supprimer '#struct-' de chaque élément de la liste
            lesrelations_cleaned = [relation.replace('#struct-', '') for relation in lesrelations]
            xml_id_cleaned = xml_id[0].lstrip('struct-') 
            
            # Organismes copubliants EX (= hors France et DOM TOM)
            if lepays and lepays[0] == pays_org:

                # print (f"{lenom} trouvé en {lepays}")
                partenaire = 1
                unique_org_ex[xml_id[0]] = {
                    "Pays_ex": lepays,  # Le pays (on filtrera ensuite)
                    "OrganismeEx": lenom[0],  # Les noms des institutions
                    "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                    "adresse": ladresse_value, 
                    "parents": lesrelations_cleaned # tutelles

                }
                
            # Organismes FR
            elif lepays and lepays[0] in France_et_dom_tom_codes:

                #print(lepays)
                unique_org_fr[xml_id[0]] = {
                    "Pays_fr": lepays,  # Le pays
                    "Organisme_fr": lenom[0],  # Les noms des institutions
                    "Acronyme_fr": lacronyme[0] if lacronyme else 'na',
                    "ID_aurehal": xml_id_cleaned,  # L'attribut xml:id
                    "adresse": ladresse_value,
                    "parents": lesrelations_cleaned

                }


        # TRAITEMENT DES NOTICES
        
        biblfull_elements = tree.findall('.//tei:biblFull', namespaces=namespaces)
        for biblfull in biblfull_elements:
            #identifiant de la publication dans HAL
            halID = biblfull.xpath('.//tei:publicationStmt/tei:idno[@type="halId"]/text()', namespaces=namespaces) or ["pas de hal_ID"]
            halID_value = halID[0] if halID else "no HalID"
            # print(halID_value)
            
            Domaines = biblfull.xpath('.//tei:profileDesc/tei:textClass/tei:classCode[@scheme="halDomain"]/text()', namespaces=namespaces)
            Domaines_value = ";".join(domaine.strip() for domaine in Domaines if domaine)

            # la valeur "Datepub" n'est pas toujours renseignée. Dans ce cas, on prend la valeur de date de production  
            date_value = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct//tei:monogr/tei:imprint/tei:date[@type="datePub"]/text()', namespaces=namespaces)
            date_produced = biblfull.xpath('.//tei:editionStmt/tei:edition/tei:date[@type="whenProduced"]/text()', namespaces=namespaces)
            if date_value and date_value is not None:
                date_text = date_value[0]  # Récupérer la chaîne de date
                year_value = date_text[:4]  # Les 4 premiers caractères pour l'année
            else:
                year_value = date_produced[0][:4]

            titre_journal = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct/tei:monogr/tei:title[@level="j"]', namespaces=namespaces)
            titre_revue = titre_journal[0].text if titre_journal  else ""
            # print(titre_revue)
            conference_titles = biblfull.xpath('.//tei:sourceDesc/tei:biblStruct/tei:monogr/tei:meeting/tei:title', namespaces=namespaces)
            titre_conf = conference_titles[0].text if conference_titles else ""
            # print(titre_conf)
    
        
        # Identification des affiliations associées à chaque auteur
            for author in biblfull.xpath('.//tei:titleStmt/tei:author', namespaces=namespaces):
                forename = author.xpath('.//tei:persName/tei:forename/text()', namespaces=namespaces)  or ["Unknown"]
                surname = author.xpath('.//tei:persName/tei:surname/text()', namespaces=namespaces)  or ["Unknown"]
                                
                authorLastFirstnames = (f"{surname[0]}, {forename[0]}")

                affiliations = author.xpath('.//tei:affiliation/@ref', namespaces=namespaces)

                for affiliation in affiliations:
                    affiliation = affiliation.lstrip('#struct-')

                    datapubli.append ({
                        "halID" : halID_value,
                        "Auteur" : authorLastFirstnames,
                        "affiliation" : affiliation,
                        "Annee" : year_value,
                        "Titre_conference" : titre_conf,
                        "Titre_revue": titre_revue,
                        "Domaines": Domaines_value,
                    })

        previous_cursor_mark = cursor_mark
        cursor_mark = next_cursor_mark
        # print(f"le prochain curseur : {cursor_mark}")
        # Pause pour éviter de surcharger l'API
        # time.sleep(0.1)

if not next_cursor_mark:
    print(compteur)
    break



CursorMark: *
1
réussi
prochain curseur :AoEnMjM3NTUwMw==
nbre résultats : 27530. Durée estimée pour 4000 notices : 20 mn
CursorMark: AoEnMjM3NTUwMw==
réussi
prochain curseur :AoEnMjQ4NDA3MA==
CursorMark: AoEnMjQ4NDA3MA==
réussi
prochain curseur :AoEnMjYxMDU1MQ==
CursorMark: AoEnMjYxMDU1MQ==
réussi
prochain curseur :AoEnMjg5NjUxNw==
CursorMark: AoEnMjg5NjUxNw==
réussi
prochain curseur :AoEnMjkzNjU2NQ==
CursorMark: AoEnMjkzNjU2NQ==
réussi
prochain curseur :AoEnMjk3NTA4NA==
CursorMark: AoEnMjk3NTA4NA==
réussi
prochain curseur :AoEnMzAyMTcxMg==
CursorMark: AoEnMzAyMTcxMg==
réussi
prochain curseur :AoEnMzA3MTczNg==
CursorMark: AoEnMzA3MTczNg==
réussi
prochain curseur :AoEnMzExMzQxNQ==
CursorMark: AoEnMzExMzQxNQ==
réussi
prochain curseur :AoEnMzEzNDk5OQ==
CursorMark: AoEnMzEzNDk5OQ==
réussi
prochain curseur :AoEnMzE1OTg3Mw==
CursorMark: AoEnMzE1OTg3Mw==
réussi
prochain curseur :AoEnMzIxNTExOA==
CursorMark: AoEnMzIxNTExOA==
réussi
prochain curseur :AoEnMzI3MTQ4NA==
CursorMark: AoEnMzI3MTQ4NA

KeyboardInterrupt: 

In [4]:

######################################################################
# Conversion en "dataframes" pour traitement des données et comptage
######################################################################

# Convertir les données collectées pour les organismes étrangers
dataex = list(unique_org_ex.values())
df_ex = pd.DataFrame(dataex)

# Convertir les données collectées pour les organismes français
datafr = list(unique_org_fr.values())
df_fr = pd.DataFrame(datafr)

# Liste des publications/logiciels de HAL
df_publis = pd.DataFrame(datapubli)


# Afficher les DataFrames
# print(df_fr)
# print(df_ex)
# print(df_publis)
                
print(f"3 df créés")


3 df créés


In [5]:
# Liste des publications (
# le nombre de halID uniques indique le nombre de notices récupérées
df_publis.describe()
# Pour contrôle : import openpyxl
# df_ex.to_excel("df_ex.xlsx", index=False)
# df_fr.to_excel("df_fr.xlsx", index=False)
# df_publis.to_excel("df_publis.xlsx", index=False)

,halID,Auteur,affiliation,Annee,Titre_conference,Titre_revue,Domaines
count,134032,134032,134032,134032,134032,134032,134032
unique,22499,38817,10342,6,6796,2299,5677
top,inserm-02914443,"Guedj, Benjamin",1039632,2022,,,Computer Science [cs]
freq,233,240,1351,32941,83064,71857,9928


In [6]:

#############################################
# FILTRE DE pour df_ex (organismes étrangers)
#############################################
# On renomme df_ex en df_DE

df_DE = df_ex
df_DE.rename(columns={"OrganismeEx" : "Organisme_DE"}, inplace=True)
df_DE.head(5)


,Pays_ex,Organisme_DE,ID_aurehal,adresse,parents
0,[DE],RWTH Aachen University = Rheinisch-Westfälisch...,303510,RWTH Aachen – Templergraben 55 – 52062 Aachen...,[]
1,[DE],Chemnitz University of Technology / Technische...,94194,Chemnitz,[]
2,[DE],Max-Planck-Institut für Informatik,54489,"Campus E-1 4, 66123 Saarbrücken, Germany",[5247]
3,[DE],Max-Planck-Gesellschaft,5247,"Jägerstrasse, 10-11, D-10117 Berlin",[]
4,[DE],Deutsches Klimarechenzentrum [Hamburg],474787,Bundesstraße 45aD-20146 Hamburg,[]


In [7]:
###########################################################
# Interprétation des codes Pays en noms en toutes lettres 
# utile si on étudie plusieurs pays
###########################################################
erreur = 0
# URL de l'API
url = "https://restcountries.com/v3.1/all"

try:
    # Récupérer les données de l'API
    response = requests.get(url)
    response.raise_for_status()  # Vérifie si la requête a échoué (code 4xx ou 5xx)
    countries_data = response.json()

    # Créer un dictionnaire code ISO 2 -> nom complet
    country_mapping = {
        country.get("cca2"): country.get("name", {}).get("common")
        for country in countries_data
        if country.get("cca2") and country.get("name")
    }

    print("✅ Données des pays récupérées avec succès !")

except requests.exceptions.RequestException as e:
    # Afficher un message en cas d'erreur
    print(f"❌ Erreur lors de la récupération des données. Veuillez réessayer : {e}")
    country_mapping = {}  # Dictionnaire vide pour éviter d'autres erreurs
    erreur = 1

# Mapper les codes pays en noms complets
for df, col, label in [(df_DE, "Pays_ex", "DE"), (df_fr, "Pays_fr", "FR")]:
    df[col] = df[col].apply(lambda x: country_mapping.get(x[0]) if isinstance(x, list) and x else x)
    df['TypePays'] = label

if erreur == 0:
    # Afficher un aperçu des DataFrames

    print(df_DE.head(1))


✅ Données des pays récupérées avec succès !
   Pays_ex                                       Organisme_DE ID_aurehal  \
0  Germany  RWTH Aachen University = Rheinisch-Westfälisch...     303510   

                                             adresse parents TypePays  
0  RWTH Aachen –  Templergraben 55 – 52062 Aachen...      []       DE  


In [8]:
##############################################################
# Liste complète des copublications avec DE par halID
# et avec les infos d'organismes co-publiants
##############################################################

#Croisement des listes d'organismes avec la liste des publications par affiliation
OrgFR_et_publis_df = ""
OrgDE_et_publis_df= ""
Orgtout_et_publis_df=""

OrgFR_et_publis_df = pd.merge(df_publis, df_fr, left_on='affiliation', right_on='ID_aurehal', how='inner')

OrgDE_et_publis_df = pd.merge(df_publis, df_DE, left_on='affiliation', right_on='ID_aurehal', how='inner')

Orgtout_et_publis_df= pd.concat([OrgFR_et_publis_df, OrgDE_et_publis_df, df_publis], ignore_index=True)




# Liste par halID unique en rassemblant les autres infos sur une seule ligne
Toutes_les_publis_df=""
Toutes_les_publis_df=Orgtout_et_publis_df.groupby(['halID']).agg({
    'Annee': lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Organisme_DE': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'adresse':lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Pays_ex': lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Organisme_fr': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Acronyme_fr': lambda x: '; '.join(x.dropna().astype(str).unique()),
    'ID_aurehal': lambda x: '; '.join(x.dropna().astype(str).unique()),
    'TypePays': lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Domaines':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Titre_revue': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Titre_conference': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
}).reset_index()

len(Toutes_les_publis_df)
Toutes_les_publis_df.head(1)

,halID,Annee,Organisme_DE,adresse,Pays_ex,Organisme_fr,Acronyme_fr,ID_aurehal,TypePays,Domaines,Titre_revue,Titre_conference
0,cea-03010533,2020,,Centre de SaclayCentre de GrenobleCentre de Ca...,,Commissariat à l'énergie atomique et aux énerg...,CEA; STORM; LIHPC; LaBRI,300016; 409747; 1055205; 3102,FR,Computer Science [cs];Computer Science [cs]/Di...,,C3PO'20 Workshop - First Workshop on Compiler-...


In [9]:
##########################################################
# Liste des organismes DE copubliants
##########################################################


#Exclusion des auteurs doublement affiliés FR dans la liste des affilitions+publications allemandes
publisDE_df=""
publisDE_df = OrgDE_et_publis_df[~OrgDE_et_publis_df.set_index(['halID', 'Auteur']).index.isin(OrgFR_et_publis_df.set_index(['halID', 'Auteur']).index)]

publisDE_df.to_excel("publisDE_df.xlsx", index=False)

In [ ]:


# On regroupe les publis_DE et les publis_FR qui ont un halID en commun
publisDE_FR_df =""
publisDE_FR_df = publisDE_df.merge(
    OrgFR_et_publis_df[["halID", "Acronyme_fr","parents"]],  # On garde uniquement les colonnes nécessaires
    on="halID",  # Clé de jointure
    how="left"  # On garde toutes les lignes de OrgDE_et_publis_df
)

# On supprime la valeur "na" qui ne correspond à rien
publisDE_FR_df["Acronyme_fr"] = publisDE_FR_df["Acronyme_fr"].replace("na", "")

# On a maintenant, en face de chaque organisme allemand, un organisme français correspondant
# Les lignes ont été doublées s'il y a 2 organismes correspondant à 2 autres (=4 lignes) et ce pour chaque auteur distinct.

# publisDE_FR_df.to_excel("publisDE_FR_df.xlsx", index=False)

publisDE_FR_df.head(5)

In [ ]:
# id_aurehal_de_la_structure = [
#     "419153", "104751", "34586", "2497", "1096051", "129671", "104752", "118511", "454310"
# ]
# # Identifier les lignes où le parent est un centre INRIA
# Ajouter la colonne 'Inria' avec "oui" si un des identifiants est dans la colonne 'parents'
publis_DE_FR_Filtre_df = publisDE_FR_df
publis_DE_FR_Filtre_df['Inria'] = publis_DE_FR_Filtre_df['parents_y'].apply(
    lambda x: 'oui' if isinstance(x, list) and any(str(id) in x for id in id_aurehal_de_la_structure) else None
)

# Garder les lignes où 'Inria' est "oui"
publis_DE_FR_Inria_df = publis_DE_FR_Filtre_df[publis_DE_FR_Filtre_df['Inria'] == 'oui']

# Ajouter la colonne "Berlin" avec "Oui" si "Berlin" ou "Potsdam" sont présents dans "Organisme_DE" ou "adresse"
publis_DE_FR_Inria_df["Berlin"] = publis_DE_FR_Inria_df.apply(
    lambda row: "Oui" if "Berlin" in str(row["Organisme_DE"]) or "Potsdam" in str(row["Organisme_DE"]) or "Berlin" in str(row["adresse"]) or "Potsdam" in str(row["adresse"]) else "Non", axis=1
)

# On ne garde que les copublis avec Berlin
publis_DE_FR_Inria_Berlin_df=publis_DE_FR_Inria_df[publis_DE_FR_Inria_df["Berlin"] == "Oui"].copy()
publis_DE_FR_Inria_Berlin_df.head(5)
# publis_DE_FR_Inria_df.to_excel("publisDE_FR_Inria.xlsx", index=False)

In [ ]:
#######################################################################
#Liste des organismes berlinois copubliant avec une structure Inria
#######################################################################

# On fait une liste par organisme DE
Copublis_DE_org = ""
Copublis_DE_org= publis_DE_FR_Inria_Berlin_df.groupby("Organisme_DE").agg({
    'Acronyme_fr':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Domaines':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Annee':  lambda x: '; '.join(x.dropna().astype(str).unique()), 
    'halID':  lambda x: '; '.join(x.dropna().astype(str).unique()),
    'ID_aurehal':  lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Pays_ex':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Titre_revue': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Titre_conference': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'adresse':lambda x: '; '.join(x.dropna().astype(str).unique()),
}).reset_index()


# Ajouter le nombre de halID uniques
halID_counts = publis_DE_FR_Inria_df.groupby("Organisme_DE")["halID"].nunique().reset_index()
halID_counts.rename(columns={"halID": "Nbre_publications"}, inplace=True)

# Fusionner avec le comptage des halID uniques
Copublis_DE_df = Copublis_DE_org.merge(halID_counts, on="Organisme_DE", how="left")



#Supprimer les ; superflus
Copublis_DE_df["Acronyme_fr"] = Copublis_DE_df["Acronyme_fr"].str.strip(";")

# Copublis_DE_df.to_excel("Copublis_DE_df.xlsx", index=False)

In [ ]:
#####################################################################
# Liste des organismes FR copubliants une structure berlinoise
#####################################################################


Copublis_FR_org = ""
Copublis_FR_org= publis_DE_FR_Inria_Berlin_df.groupby("Acronyme_fr").agg({
    'Organisme_DE':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Domaines':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'halID':  lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Annee':  lambda x: '; '.join(x.dropna().astype(str).unique()), 
    'Titre_revue': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Titre_conference': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'adresse':lambda x: '; '.join(x.dropna().astype(str).unique()),
}).reset_index()


# Ajouter le nombre de halID uniques
halID_counts = publis_DE_FR_Inria_df.groupby("Acronyme_fr")["halID"].nunique().reset_index()
halID_counts.rename(columns={"halID": "Nbre_publications"}, inplace=True)

# Fusionner avec le comptage des halID uniques
Copublis_FR_df = Copublis_FR_org.merge(halID_counts, on="Acronyme_fr", how="left")


# Copublis_FR_df.to_excel("Copublis_FR_df.xlsx", index=False)




In [30]:
##################################
# Liste des publications par HalID
##################################


#On garde les Berlinois d'un côté
Toutes_les_copublis_DE_FR_Berlin = ""
Toutes_les_copublis_DE_FR_Berlin=publis_DE_FR_Inria_df[publis_DE_FR_Inria_df["Berlin"] == "Oui"].copy()
Toutes_les_copublis_DE_FR_Berlin= Toutes_les_copublis_DE_FR_Berlin.groupby("halID").agg({
    'Organisme_DE':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Acronyme_fr':  lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Domaines':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Annee':  lambda x: '; '.join(x.dropna().astype(str).unique()), 
    'Titre_revue': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Titre_conference': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'adresse':lambda x: '; '.join(x.dropna().astype(str).unique()),
}).reset_index()



#On garde les non Berlinois de l'autre
Toutes_les_copublis_DE_FR_Hors_Berlin=""
Toutes_les_copublis_DE_FR_Hors_Berlin=publis_DE_FR_Inria_df[publis_DE_FR_Inria_df["Berlin"] == "Non"].copy()
Toutes_les_copublis_DE_FR_Hors_Berlin= Toutes_les_copublis_DE_FR_Hors_Berlin.groupby("halID").agg({
    'Organisme_DE':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Acronyme_fr':  lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Domaines':lambda x: '; '.join(x.dropna().astype(str).unique()),
    'Annee':  lambda x: '; '.join(x.dropna().astype(str).unique()), 
    'Titre_revue': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'Titre_conference': lambda x: '; '.join(x.dropna().astype(str).unique()[x.dropna().astype(str).unique() != ""]),
    'adresse':lambda x: '; '.join(x.dropna().astype(str).unique()),
}).reset_index()
# Toutes_les_copublis_DE_FR_Hors_Berlin.to_excel("Toutes_les_copublis_DE_FR_Hors_Berlin.xlsx", index=False)


In [ ]:
###########################################################
# Liste des Organismes berlinois + domaines + org FR et nombre de publis
###########################################################
# On se base sur publis_DE_FR_Inria_Berlin_df = la liste des publications communes à Inria et à des organismes berlinois.
# On va "explode" les domaines pour en avoir un par ligne
# on supprime les sous-domaines, on ne garde que les grands domaines


# On sépare les domaines en lignes distinctes
df_exploded = ""
df_exploded = publis_DE_FR_Inria_Berlin_df.assign(Domaines=publis_DE_FR_Inria_Berlin_df["Domaines"].str.split(";")).explode("Domaines")
df_exploded.head(2)



# On supprime les sous-domaines
df_exploded = df_exploded[~df_exploded["Domaines"].str.contains("/", na=False)]
df_exploded.describe()
# df_exploded.to_excel("df_grands_domaines.xlsx", index=False)



domain_counts = ""
domain_counts = (
    df_exploded.groupby(["Organisme_DE", "Domaines"])
    .agg(
        Acronymes_FR=("Acronyme_fr", lambda x: ";".join(x.unique())),  # Agrège les acronymes français uniques
        halID=("halID", lambda x: ";".join(x.unique())),  # Agrège les acronymes français uniques
    )
    .reset_index()
)



domain_counts["Count_halID"] = domain_counts["halID"].str.count(";").fillna(0).astype(int) + 1


# domain_counts.to_excel("Nombre_de_domaines_par_orgv2.xlsx", index=False)



In [26]:
##################################################################
# Liste des Domaines avec liste des organismes correspondants
# Regroupement par Domaine  et lister les organismes DE, les acronymes, les halID et compter ces derniers
##################################################################

Domain_listOrg_counts = ""
Domain_listOrg_counts = (
    df_exploded.groupby(["Domaines"])
    .agg(
        Organisme_DE =("Organisme_DE", lambda x: ";".join(x.unique())),
        Acronymes_FR=("Acronyme_fr", lambda x: ";".join(x.unique())),  # Agrège les acronymes français uniques
        halID=("halID", lambda x: ";".join(x.unique())),  # Agrège les acronymes français uniques
    )
    .reset_index()
)

Domain_listOrg_counts["Count_halID"] = Domain_listOrg_counts["halID"].str.count(";").fillna(0).astype(int) + 1
# Domain_listOrg_counts.to_excel("Liste_des_domaines.xlsx", index=False)



# Génération du fichier Excel

In [ ]:
##########################################
# Export excel des listes obtenues
##########################################

# Chemin du fichier Excel à créer
fichier_excel = f"Copublications_INRIA_DE_2020-2024.xlsx"

# Méthodologie pour le premier onglet du fichier
methodologie_markdown = """
# Collection: copublications Inria / Organismes à Berlin (Allemagne)
* demande interne Inria (27/03/2025)
* Réalisation du script : Kumar Guha (Data DCIS/Inria)
* Date 31/03/2025

## Choix
* On ne retient que la première affiliation de chaque auteur (pas les niveaux supérieurs : exemple : Boston University School of Medicine et pas Boston University).
* Si un auteur rattaché à une structure française est aussi rattaché à une structure étrangère, on ne retient pas cette structure étrangère dans la décompte des copubliants étrangers. 
* Les publications où les auteurs sont affiliés à des structures ayant pour tutelle un centre Inria sont comptées comme des copublications FR.
* Organisme à Berlin : toute structure dont le nom ou l'adresse contiennent les termes "Berlin" ou "Potsdam".

## Limites et solution adoptée
* Les structures présentes dans le référentiel Aurehal ne sont pas toujours bien renseignées (ville non mentionnée, adresse manquante), il y a peut-être des organismes situés à Berlin qui n'entreront donc pas dans le résultat. Solution : une liste des organismes allemands mais pas identifiés comme étant à Berlin est ajoutée au résultat.

## Étapes
* Extraire les publications des équipes concernées et récupérer les informations de licence.
* identifier les publications dont les auteurs sont affiliés à un organisme étranger (hors France et DOM TOM)
* On crée des listes d'identifiants uniques pour les affiliations FR et DE, on filtrera ensuite sur Berlin.
    *  on exclut les organismes étrangers dont les auteurs sont aussi affiliés à une structure FR
    *  on exclut les affiliations en double pour une même publication

## Résultat
* Génération d'un fichier Excel avec : 
    * Liste des copublications avec un organisme berlinois
    * Liste des organismes berlinois copubliants, et des domaines
    * Liste des équipes INRIA copubliantes
    * Liste des domaines des publications et des organismes berlinois copubliants
    * Liste des autres organismes allemands copubliants

"""

# Création du fichier Excel avec l'ordre des onglets
with pd.ExcelWriter(fichier_excel, engine='xlsxwriter') as writer:
    # Ajouter l'onglet "Méthodologie" en premier
    workbook = writer.book
    worksheet = workbook.add_worksheet("Méthodologie")
    writer.sheets["Méthodologie"] = worksheet

    # Insérer le texte Markdown dans la cellule A1
    worksheet.write(0, 0, methodologie_markdown)

    # Ajuster la largeur de la colonne A
    worksheet.set_column(0, 0, 118)

    # Activer le retour à la ligne dans la cellule A1
    cell_format = workbook.add_format({'text_wrap': True})
    worksheet.set_row(0, None, cell_format)  # Appliquer le format à la ligne 1

    # Ajouter les autres onglets

    
    Toutes_les_copublis_DE_FR_Berlin.to_excel(writer, sheet_name="Copublication_avec_Berlin", index=False) 

    Copublis_DE_df.to_excel(writer, sheet_name="Org_Berlin_Org_FR", index=False) 
    
    Copublis_FR_df.to_excel(writer, sheet_name="EquipesInria_et_org_Berlin", index=False) 
    
    domain_counts.to_excel(writer, sheet_name="Org_Berlin_et_domaines", index=False)

    Domain_listOrg_counts.to_excel(writer, sheet_name="Domaines", index=False) # index=false, on n'aura pas de colonne avec numérotation de lignes
    
    Toutes_les_copublis_DE_FR_Hors_Berlin.to_excel(writer, sheet_name="CopublicationsDE_horsBerlin", index=False) 


############## ajustement largeur de colonnes titres courts #####################

    # Liste des feuilles et des colonnes à ajuster
    sheets = {
        "Copublication_avec_Berlin": ["B","C","D"],
        "Org_Berlin_Org_FR": ["A","B"],  
        "Org_Berlin_et_domaines": ["A","B"], 
        "EquipesInria_et_org_Berlin" :["B"],
        "Domaines": ["A","B"],
        "CopublicationsDE_horsBerlin": ["B","C","D"],   
    }
    # Largeur des colonnes à ajuster
    column_width = 40  # Ajuste selon besoin

    # Appliquer le format
    for sheet_name, columns in sheets.items():
        worksheet = writer.sheets[sheet_name]
        for col in columns:
            worksheet.set_column(f"{col}:{col}", column_width)

# ############## ajustement largeur de colonnes titres courts #####################
    # Liste des feuilles et des colonnes à ajuster
    sheets2 = {
        "Copublication_avec_Berlin": ["A","D"],  
        "Org_Berlin_Org_FR": ["C","E"],
        "EquipesInria_et_org_Berlin" :["A","D"],
        "CopublicationsDE_horsBerlin" : ["A"]
    }

    # Largeur des colonnes à ajuster
    column_width2 = 20  # Ajuste selon besoin

    # Appliquer le format
    for sheet_name, columns in sheets2.items():
        worksheet = writer.sheets[sheet_name]
        for col in columns:
            worksheet.set_column(f"{col}:{col}", column_width2)

# Message de confirmation
print(f"Fichier Excel '{fichier_excel}' créé.")

Fichier Excel 'Copublications_INRIA_DE_2020-2024.xlsx' créé.


# dédoublonner copublications avec Berlin et Hors Berlin
# modifier la largeur des colonnes

In [34]:
## Rendre la colonne HAL_ID () cliquable dans fichier excel

from openpyxl import load_workbook


# Charger le fichier Excel
wb = load_workbook(fichier_excel)

# Sélectionner l'onglet où on veut ajouter les liens
ws = wb["Liste des publications"]  # Nom de l'onglet cible


# Définir une fonction pour créer les liens
def create_link_hal(cellule):
    return f'https://inria.hal.science/{hal}'

def create_link_aurehal(cellule):
    return f'https://aurehal.archives-ouvertes.fr/structure/read/id/{aurehal}'


# Parcourir les cellules de la colonne 'ID équipe de recherche' et ajouter les liens hypertexte
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1):
    for cell in row:
        hal = cell.value
        if hal:
            cell.hyperlink = create_link_hal(hal)


# Sélectionner l'onglet où on veut ajouter les liens
ws = wb["Organismes DE"]  # Nom de l'onglet cible
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=6, max_col=6):
    for cell in row:
        aurehal = cell.value
        if aurehal:
            cell.hyperlink = create_link_aurehal(aurehal)


# ws = wb["Organismes hors UE"]  # Nom de l'onglet cible
# for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=4, max_col=4):
#     for cell in row:
#         aurehal = cell.value
#         if aurehal:
#             cell.hyperlink = create_link_aurehal(aurehal)

ws = wb["Organismes FR"]  # Nom de l'onglet cible
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=5, max_col=5):
    for cell in row:
        aurehal = cell.value
        if aurehal:
            cell.hyperlink = create_link_aurehal(aurehal)

print ("liens hal/Aurehal terminés dans les onglets")




# Enregistrer les modifications dans le fichier Excel
wb.save(fichier_excel)

print(f"Fichier {fichier_excel} finalisé ")

liens hal/Aurehal terminés dans les onglets
Fichier Copublications_INRIA_DE_2020-2024.xlsx finalisé 
